In [1]:
import sys
sys.path.append("src")
import os

In [2]:
import libfeatureselection
import libpybiofeature
from libmodel import t3mm

2023-07-06 21:33:16.382718: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-06 21:33:16.384969: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-06 21:33:16.384977: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-06 21:33:16.919313: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-06 21:33:16.919332: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

In [19]:
import numpy as np
import pandas as pd
import json
from Bio import SeqIO

In [4]:
import utils
work_Dir = utils.workdir.workdir(os.getcwd(), 4)
work_Dir

'/mnt/md0/Users/georgezhao/Source/TxSEml_Addon'

In [5]:
protype, cter_bool, db_size = 2, False, 'small'
Tx_arg = {
    "type": f'T{protype}',
    'seq_id': os.path.join(work_Dir, *['data', 'db', f'T{protype}', 'seq_id.json']),
    'shufflesplit_index_file': os.path.join(work_Dir, *['data', 'db', f'T{protype}', 'seq_id_shufflesplit.json']),
    'fasta': {
        'cter': cter_bool,
        'sp': {
            'p': "data/T2SE/spT2SE.fasta",
            'n': "data/T2SE/sp_paired_non_t2se.fasta"
        },
        'nosp': {
            'p': "data/T2SE/nospT2SE.fasta",
            'n': "data/T2SE/nosp_paired_non_t2se.fasta"
        },
    },
    'expasy': {
        'sp': {
            'p': "out/libfeatureselection/SP_feature_research/featuredb/spT2SE_expasy.json",
            'n': "out/libfeatureselection/SP_feature_research/featuredb/sp_paired_non_t2se_expasy.json"
        },
        'nosp': {
            'p': "out/libfeatureselection/SP_feature_research/featuredb/nospT2SE_expasy.json",
            'n': "out/libfeatureselection/SP_feature_research/featuredb/nosp_paired_non_t2se_expasy.json"
        },
    },
    'bliulab': {
        'PC': {
            'sp': {
                'p': "out/libfeatureselection/SP_feature_research/featuredb/spT2SE_PCPseAAC.csv",
                'n': "out/libfeatureselection/SP_feature_research/featuredb/sp_paired_non_t2se_PCPseAAC.csv",
            },
            'nosp': {
                'p': "out/libfeatureselection/SP_feature_research/featuredb/nospT2SE_PCPseAAC.csv",
                'n': "out/libfeatureselection/SP_feature_research/featuredb/nosp_paired_non_t2se_PCPseAAC.csv"
            },
        },
        'Topm': {
            'sp': {
                'p': "out/libfeatureselection/SP_feature_research/featuredb/spT2SE_topm.json",
                'n': "out/libfeatureselection/SP_feature_research/featuredb/sp_paired_non_t2se_topm.json",
            },
            'nosp': {
                'p': "out/libfeatureselection/SP_feature_research/featuredb/nospT2SE_topm.json",
                'n': "out/libfeatureselection/SP_feature_research/featuredb/nosp_paired_non_t2se_topm.json"
            },
        },
    },
    "ss": {
        'sp': {
            'p': "out/libfeatureselection/SP_feature_research/featuredb/bert/sp_p_ss.pkl",
            'n': "out/libfeatureselection/SP_feature_research/featuredb/bert/sp_n_ss.pkl",
        },
        'nosp': {
            'p': "out/libfeatureselection/SP_feature_research/featuredb/bert/nosp_p_ss.pkl",
            'n': "out/libfeatureselection/SP_feature_research/featuredb/bert/nosp_n_ss.pkl",
        },
    },
    "diso": {
        'sp': {
            'p': "out/libfeatureselection/SP_feature_research/featuredb/bert/sp_p_diso.pkl",
            'n': "out/libfeatureselection/SP_feature_research/featuredb/bert/sp_n_diso.pkl",
        },
        'nosp': {
            'p': "out/libfeatureselection/SP_feature_research/featuredb/bert/nosp_p_diso.pkl",
            'n': "out/libfeatureselection/SP_feature_research/featuredb/bert/nosp_n_diso.pkl",
        },
    },
    'a': {
        'sp': {
            "p": "out/libfeatureselection/SP_feature_research/featuredb/aligned_t2_spT2SE.json",
            "n": "out/libfeatureselection/SP_feature_research/featuredb/aligned_t2_sp_paired_non_t2se.json",
        },
        'nosp': {
            "p": "out/libfeatureselection/SP_feature_research/featuredb/aligned_t2_nospT2SE.json",
            "n": "out/libfeatureselection/SP_feature_research/featuredb/aligned_t2_nosp_paired_non_t2se.json",
        }
    },
    'possum': {
        'index': "out/libfeatureselection/SP_feature_research/featuredb/possum/possum_index.json",
        'pssm_fdb_pattern': "out/libfeatureselection/SP_feature_research/featuredb/possum/{zipid}_pssm_features.zip",
        'pssm_rdb_pattern': "out/libfeatureselection/SP_feature_research/featuredb/possum/{zipid}_pssm_files.zip"
    },
    'model': {
        'size': db_size,
        'cter': cter_bool,
        "path_to_save_dir": f"out/libfeatureselection/SP_feature_research/model/"
    }
}

In [6]:
seq_id_dict = {
    "sp": {
        "p": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['sp']['p'], "fasta") ],
        "n": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['sp']['n'], "fasta") ],
    },
    "nosp": {
        "p": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['nosp']['p'], "fasta") ],
        "n": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['nosp']['n'], "fasta") ],
    },
}

In [7]:
feature_data_set = []

In [8]:
# BPBaac
BPBaac_seq_data = {
    "sp_p": libpybiofeature.libdataloader.fasta_seq_loader.prepare_data(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
    )[0].values.tolist(),
    "sp_n": libpybiofeature.libdataloader.fasta_seq_loader.prepare_data(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
    )[0].values.tolist(),
    "nosp_p": libpybiofeature.libdataloader.fasta_seq_loader.prepare_data(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
    )[0].values.tolist(),
    "nosp_n": libpybiofeature.libdataloader.fasta_seq_loader.prepare_data(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
    )[0].values.tolist(),
}

BPBaac_profile = {
    "p": libpybiofeature.BPBaac_psp.mat_constructor(
        fasta_db=BPBaac_seq_data['sp_p'],
        cter=Tx_arg['model']['cter'],
        terlength=100,
        padding_ac='A'
    ),
    "n": libpybiofeature.BPBaac_psp.mat_constructor(
        fasta_db=BPBaac_seq_data['sp_n'],
        cter=Tx_arg['model']['cter'],
        terlength=100,
        padding_ac='A'
    ),
}

with open(f"{Tx_arg['model']['path_to_save_dir']}/BPBaac_profile.json", "w+", encoding='UTF-8') as f:
    json.dump(BPBaac_profile, f)

for data_type in BPBaac_seq_data.keys():
    BPBaac_seq_data[data_type] = pd.DataFrame(
        [
            libpybiofeature.BPBaac_psp.mat_mapper(
                seq=str(seq.seq),
                pmat=BPBaac_profile['p'],
                nmat=BPBaac_profile['n'],
                cter=Tx_arg['model']['cter'],
                terlength=100,
                padding_ac='A'
            ) for seq in BPBaac_seq_data[data_type]
        ],
        index=seq_id_dict[data_type.split("_")[0]][data_type.split("_")[1]]
    )
BPBaac_seq_data['name'] = "BPBaac"
feature_data_set.append(BPBaac_seq_data)

In [9]:
possum_index_dict = None
with open(Tx_arg['possum']['index'], 'r', encoding='UTF-8') as f:
    possum_index_dict = json.load(f)

feature_data_set.append({
    "name": "PSSM_feature",
    "sp_p": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['sp_p'],
        feature_name_list=['rpm_pssm', "d_fpssm", 'tpc', 's_fpssm'],
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['sp']['p'], :],
    "sp_n": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['sp_n'],
        feature_name_list=['rpm_pssm', "d_fpssm", 'tpc', 's_fpssm'],
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['sp']['n'], :],
    "nosp_p": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['nosp_p'],
        feature_name_list=['rpm_pssm', "d_fpssm", 'tpc', 's_fpssm'],
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['nosp']['p'], :],
    "nosp_n": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['nosp_n'],
        feature_name_list=['rpm_pssm', "d_fpssm", 'tpc', 's_fpssm'],
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['nosp']['n'], :],
})

In [10]:
# Expasy PP
feature_data_set.append({
    "name": "Expasy_PP",
    "sp_p": libpybiofeature.libdataloader.expasy.get_expasy_t3sps(
        path_to_json=Tx_arg['expasy']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p']
    ),
    "sp_n": libpybiofeature.libdataloader.expasy.get_expasy_t3sps(
        path_to_json=Tx_arg['expasy']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n']
    ),
    "nosp_p": libpybiofeature.libdataloader.expasy.get_expasy_t3sps(
        path_to_json=Tx_arg['expasy']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p']
    ),
    "nosp_n": libpybiofeature.libdataloader.expasy.get_expasy_t3sps(
        path_to_json=Tx_arg['expasy']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n']
    )
})

nosp_paired_non_t2se_expasy.json_100_False: 100%|██████████| 11/11 [00:00<00:00, 286567.35it/s]


In [11]:
feature_data_set.append({
    "name": "Top_n_gram",
    "sp_p": libpybiofeature.libdataloader.bliulab_new.Topm(
        path_to_Topm=Tx_arg["bliulab"]['Topm']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p']
    ),
    "sp_n": libpybiofeature.libdataloader.bliulab_new.Topm(
        path_to_Topm=Tx_arg["bliulab"]['Topm']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n']
    ),
    "nosp_p": libpybiofeature.libdataloader.bliulab_new.Topm(
        path_to_Topm=Tx_arg["bliulab"]['Topm']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p']
    ),
    "nosp_n": libpybiofeature.libdataloader.bliulab_new.Topm(
        path_to_Topm=Tx_arg["bliulab"]['Topm']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n']
    )
})

In [12]:
def read_PC_data(
    path_to_csv: str,
    path_to_fasta: str,
):
    df = pd.read_csv(path_to_csv, index_col=None, header=None)
    df.index = [
        seq.id for seq in SeqIO.parse(path_to_fasta, "fasta")
    ]
    df.columns = list("ACDEFGHIKLMNPQRSTVWY") + [
        str(i) for i in range(1, (df.shape[1] - 20) + 1)
    ]
    return df

feature_data_set.append({
    "name": "PCPseAAC",
    "sp_p": read_PC_data(
        path_to_csv=Tx_arg["bliulab"]['PC']['sp']['p'],
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
    ),
    "sp_n": read_PC_data(
        path_to_csv=Tx_arg["bliulab"]['PC']['sp']['n'],
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
    ),
    "nosp_p": read_PC_data(
        path_to_csv=Tx_arg["bliulab"]['PC']['nosp']['p'],
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
    ),
    "nosp_n": read_PC_data(
        path_to_csv=Tx_arg["bliulab"]['PC']['nosp']['n'],
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
    )
})

In [13]:
feature_data_set.append({
    "name": "SS-100-AC",
    "sp_p": libfeatureselection.feature_loader.load_from_bert_aac(
        path=Tx_arg['ss']['sp']['p'],
        id_list=seq_id_dict['sp']['p'],
        dim=3
    ),
    "sp_n": libfeatureselection.feature_loader.load_from_bert_aac(
        path=Tx_arg['ss']['sp']['n'],
        id_list=seq_id_dict['sp']['n'],
        dim=3
    ),
    "nosp_p": libfeatureselection.feature_loader.load_from_bert_aac(
        path=Tx_arg['ss']['nosp']['p'],
        id_list=seq_id_dict['nosp']['p'],
        dim=3
    ),
    "nosp_n": libfeatureselection.feature_loader.load_from_bert_aac(
        path=Tx_arg['ss']['nosp']['n'],
        id_list=seq_id_dict['nosp']['n'],
        dim=3
    )
})

In [14]:
feature_data_set.append({
    "name": "DISO-100-DC",
    "sp_p": libfeatureselection.feature_loader.load_from_bert_dac(
        path=Tx_arg['diso']['sp']['p'],
        id_list=seq_id_dict['sp']['p'],
        dim=2
    ),
    "sp_n": libfeatureselection.feature_loader.load_from_bert_dac(
        path=Tx_arg['diso']['sp']['n'],
        id_list=seq_id_dict['sp']['n'],
        dim=2
    ),
    "nosp_p": libfeatureselection.feature_loader.load_from_bert_dac(
        path=Tx_arg['diso']['nosp']['p'],
        id_list=seq_id_dict['nosp']['p'],
        dim=2
    ),
    "nosp_n": libfeatureselection.feature_loader.load_from_bert_dac(
        path=Tx_arg['diso']['nosp']['n'],
        id_list=seq_id_dict['nosp']['n'],
        dim=2
    )
})

In [15]:
a45_set = {
    "name": "Similarity_BLOSUM45",
}
_, _, a45_set['sp_p'] = libpybiofeature.libdataloader.bliulab_new.build_form_of_data_set_align(
    path_to_align_result=Tx_arg['a']['sp']['p'],
    seq_id_list=seq_id_dict['sp']['p'],
)
_, _, a45_set['sp_n'] = libpybiofeature.libdataloader.bliulab_new.build_form_of_data_set_align(
    path_to_align_result=Tx_arg['a']['sp']['n'],
    seq_id_list=seq_id_dict['sp']['n'],
)
_, _, a45_set['nosp_p'] = libpybiofeature.libdataloader.bliulab_new.build_form_of_data_set_align(
    path_to_align_result=Tx_arg['a']['nosp']['p'],
    seq_id_list=seq_id_dict['nosp']['p'],
)
_, _, a45_set['nosp_n'] = libpybiofeature.libdataloader.bliulab_new.build_form_of_data_set_align(
    path_to_align_result=Tx_arg['a']['nosp']['n'],
    seq_id_list=seq_id_dict['nosp']['n'],
)
feature_data_set.append(a45_set)

In [17]:
feature_data_set.append({
    "name": "CKSAAP",
    "sp_p": libpybiofeature.featurebuilder.build_CKSAAP_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p'
    ),
    "sp_n": libpybiofeature.featurebuilder.build_CKSAAP_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n'
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_CKSAAP_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p'
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_CKSAAP_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n'
    )
})

nosp_n_CKSAAP: 100%|██████████| 11/11 [00:00<00:00, 2351.07it/s]


In [18]:
feature_data_set.append({
    "name": "QSO",
    "sp_p": libpybiofeature.featurebuilder.build_qso_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p',
        cter=Tx_arg['model']['cter']
    ),
    "sp_n": libpybiofeature.featurebuilder.build_qso_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n',
        cter=Tx_arg['model']['cter']
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_qso_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p',
        cter=Tx_arg['model']['cter']
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_qso_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n',
        cter=Tx_arg['model']['cter']
    )
})

nosp_n_QSO: 100%|██████████| 11/11 [00:00<00:00, 277.00it/s]


In [20]:
def merge_pn_dataset(
    p_f: pd.DataFrame,
    p_l: np.ndarray,
    n_f: pd.DataFrame,
    n_l: np.ndarray
):
    t_f = pd.concat([p_f, n_f])
    t_l = np.concatenate([p_l, n_l])

    return t_f, t_l

# 合并feature_selected成数据集
data_set_split = {
    datatype: pd.concat([
        item[datatype] for item in feature_data_set
    ], axis=1)
    for datatype in ["sp_p", "sp_n", "nosp_p", "nosp_n"]
}
label_set_split = {
    datatype: np.ones(
        shape=(feature_data_set[0][datatype].shape[0], ))
    for datatype in ["sp_p", "nosp_p",]
} | {
    datatype: np.zeros(
        shape=(feature_data_set[0][datatype].shape[0], ))
    for datatype in ["sp_n", "nosp_n",]
}

t_f, t_l = merge_pn_dataset(
    p_f=data_set_split["sp_p"],
    p_l=label_set_split["sp_p"],
    n_f=data_set_split["sp_n"],
    n_l=label_set_split["sp_n"],
)
v_f, v_l = merge_pn_dataset(
    p_f=data_set_split["nosp_p"],
    p_l=label_set_split["nosp_p"],
    n_f=data_set_split["nosp_n"],
    n_l=label_set_split["nosp_n"],
)

In [25]:
# 丢入model中进行训练
model_path = f"{Tx_arg['model']['path_to_save_dir']}/155100983136298/"
os.makedirs(
    model_path,
    exist_ok=True
)
model_path

'out/libfeatureselection/SP_feature_research/model//155100983136298/'

In [26]:
from libfeatureselection import model, model_space
n_jobs = (
    (os.cpu_count() - 2)
    if "n_jobs" not in os.environ or os.environ['n_jobs'] == "" else
    int(os.environ['n_jobs'])
)
n_jobs

30

In [29]:
search_result_in_a_scheme_df = pd.DataFrame()
for model_index in range(len(model_space.find_space)):
    model_information_summary, searched_result_performance_summary, searched_result_5C_performance_summary = model.MyOptimitzer(
        classifier_name=model_space.find_space[model_index]['name'],
        classifier_class=model_space.find_space[model_index]['class'],
        classifier_param_dict=model_space.find_space[model_index]['param'],
    ).find_best(
        X=t_f.values,
        y=t_l,
        validation=(
            v_f.values,
            v_l
        ),
        search_method=(
            "BayesSearchCV"
            if "Bayes" not in model_space.find_space[model_index]
            or model_space.find_space[model_index]['Bayes'] == True
            else "GridSearchCV"
        ),
        n_jobs=n_jobs
    ).get_summary(
        path_to_dir=model_path
    )

    # 记录结果，插入到 search_result_in_a_scheme_df
    result_series = pd.concat([
        pd.Series(model_information_summary),
        pd.Series(searched_result_performance_summary),
        pd.Series(searched_result_5C_performance_summary),
    ], keys=[
        "Model_Information",
        "Best_Performance",
        "5FoldCV_Performance",
    ])

    result_series.name = model_index

    search_result_in_a_scheme_df = pd.concat([
        search_result_in_a_scheme_df,
        result_series.to_frame().T
    ], axis=0, ignore_index=False)

    search_result_in_a_scheme_df.index = search_result_in_a_scheme_df.index.set_names(
        ["Model_Type",]
    )

    local_xlsx_path = f"{model_path}/searched_result.xlsx"

    # 缓存 search_result_in_a_scheme_df
    search_result_in_a_scheme_df.to_excel(
        local_xlsx_path,
        "RTX-model_Predict_non-RTX",
        freeze_panes=(2, 1)
    )

/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 219, in __call__
    return self._score(
  File "/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 352, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/sklearn/model_selection/_validatio

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

...

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>